In [1]:
import pandas as pd
import numpy as np
import geopandas as gpd
import h5py
import boto.s3
import glob
import boto3
from zipfile import ZipFile

#### Juan's Code

In [2]:
s3 = boto3.client("s3")

In [ ]:
#Download File from s3 bucket 
key = "pilates-outputs/sfbay-base-20220409/activitysim/year-2018-iteration-5/trip_mode_choice.zip"
s3.download_file(
    Bucket="beam-outputs", Key=key, Filename="trip_mode_choice.zip")

In [ ]:
#Automatically extracting ZipFile with a python command 
with ZipFile('trip_mode_choice.zip', 'r') as zipObj:
   # Extract all the contents of zip file in current directory
   zipObj.extractall()

#### SF 2018

In [2]:
# File location on S3 (The address should be updated depending on the version of the code using)
loc_2018_baseline = "https://beam-outputs.s3.amazonaws.com/pilates-outputs/sfbay-base-20220409/beam/year-2018-iteration-5/ITERS/it.0/"
loc_2018_transitFreq05 = "https://beam-outputs.s3.amazonaws.com/pilates-outputs/sfbay-transit_frequencies_0.5-20220528/beam/year-2018-iteration-5/ITERS/it.0/"
loc_2018_transitFreq1p5 = "https://beam-outputs.s3.amazonaws.com/pilates-outputs/sfbay-transit_frequencies_1.5-20220529/beam/year-2018-iteration-5/ITERS/it.0/"
loc_2018_transitFreq2 = "https://beam-outputs.s3.amazonaws.com/pilates-outputs/sfbay-transit_frequencies_2.0-20220529/beam/year-2018-iteration-5/ITERS/it.0/"
loc_2018_rhFleetsz0125 = "https://beam-outputs.s3.amazonaws.com/pilates-outputs/sfbay-RH_fleetsz_0.125-20220408/beam/year-2018-iteration-5/ITERS/it.0/"
loc_2018_rhFleetsz025 = "https://beam-outputs.s3.amazonaws.com/pilates-outputs/sfbay-RH_fleetsz_0.25-20220408/beam/year-2018-iteration-5/ITERS/it.0/"
loc_2018_rhFleetsz05 = "https://beam-outputs.s3.amazonaws.com/pilates-outputs/sfbay-RH_fleetsz_0.5-20220408/beam/year-2018-iteration-5/ITERS/it.0/"
loc_2018_rhFleetsz150 = "https://beam-outputs.s3.amazonaws.com/pilates-outputs/sfbay-RH_fleetsz_1.5-20220328/beam/year-2018-iteration-5/ITERS/it.0/"
loc_2018_rhFleetsz175 = "https://beam-outputs.s3.amazonaws.com/pilates-outputs/sfbay-RH_fleetsz_1.75-20220408/beam/year-2018-iteration-5/ITERS/it.0/"

#### SF 2019

In [4]:
loc_2019_transCap05 = "https://beam-outputs.s3.amazonaws.com/pilates-outputs/sfbay-transit_capacity_0.5-20220329/beam/year-2019-iteration-5/ITERS/it.0/"

In [3]:
%%time
# Reading the events file
dtypes = {
    "time": "float32",
    "type": "category",
    "legMode": "category",
    "actType": "category", 
    "primaryFuelLevel": "float64",
    "legMode": "category",
    "chargingPointType":"category",
    "pricingModel":"category",
    "parkingType":"category",
    "mode":"category",
    "personalVehicleAvailable": "category",
    "person": "object",
    "driver": "object",
    "riders": "object",
    'primaryFuelType': "category",
    'secondaryFuelType': 'category',
    'currentTourMode': 'category',
    'currentActivity': 'category',
    'nextActivity': 'category'    
}
# Use list comprehension to remove the unwanted column in **usecol**
eventsSF = pd.read_csv(loc_2018_baseline + '0.events.csv.gz', compression = 'gzip', dtype = dtypes)

<timed exec>:24: DtypeWarning: Columns (7,12,13,17,23,29,32,44,53) have mixed types. Specify dtype option on import or set low_memory=False.


CPU times: total: 3min 8s
Wall time: 3min 47s


In [4]:
# Show all columns and rows
pd.options.display.max_columns = None
pd.options.display.max_rows = None

In [10]:
eventsSF[(eventsSF['tollPaid'] == 5)|(eventsSF['tollCost'] == 5)]

,person,vehicle,time,type,mode,currentTourMode,expectedMaximumUtility,availableAlternatives,location,personalVehicleAvailable,length,tourIndex,legModes,legVehicleIds,currentActivity,nextActivity,tripId,reason,link,legMode,facility,actType,shiftStatus,parkingTaz,chargingPointType,pricingModel,parkingType,locationY,locationX,parkingZoneId,fuel,duration,vehicleType,price,endY,endX,startY,startX,arrivalTime,departureTime,primaryFuelLevel,secondaryFuelLevel,cost,driver,links,numPassengers,primaryFuel,riders,toStopIndex,fromStopIndex,seatingCapacity,tollPaid,capacity,linkTravelTime,secondaryFuel,secondaryFuelType,primaryFuelType,departTime,score,incentive,tollCost,netCost
1280068,NaN,640215,23777.0,PathTraversal,car,car,NaN,NaN,NaN,NaN,9998.156,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,conv-L1-10000-to-25000-LowTech-2019,NaN,37.446779,-122.171455,37.388450,-122.088088,23777.0,23169.0,3.603471e+09,0.0,NaN,2228872,"502,20166,176724,20179,29318,60484,173028,8747...",0.0,1.354192e+07,NaN,NaN,NaN,3.0,5.0,3.0,"3.547,1.481,17.421,3.316,3.715,18.845,1.665,0....",0.0,None,Gasoline,NaN,NaN,NaN,NaN,NaN
1295781,2228872,NaN,23797.0,PersonCost,car,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.0,5.0,6.121612
26617741,NaN,rideHailVehicle-1329595@default,64288.0,PathTraversal,car,NaN,NaN,NaN,NaN,NaN,6354.214,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,conv-L1-75000-to-100000-LowTech-2019,NaN,37.388164,-122.088614,37.427862,-122.107909,64288.0,63904.0,3.645221e+09,0.0,NaN,rideHailAgent-1329595,"44927,42741,2289,44936,52226,175704,175254,857...",1.0,1.068810e+07,3061270,NaN,NaN,3.0,5.0,3.0,"1.223,63.363,5.003,9.273,4.275,3.749,11.489,0....",0.0,None,Gasoline,NaN,NaN,NaN,NaN,NaN
27093722,NaN,rideHailVehicle-2982843@default,64832.0,PathTraversal,car,NaN,NaN,NaN,NaN,NaN,505.113,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,conv-L1-100000-to-200000-LowTech-2019,NaN,37.388164,-122.088614,37.389558,-122.091434,64832.0,64800.0,3.547992e+09,0.0,NaN,rideHailAgent-2982843,"20176,20180,173070,83001,20170,507,505,502",0.0,7.857153e+05,NaN,NaN,NaN,3.0,5.0,3.0,"3.053,1.603,16.129,5.001,1.585,1.263,3.307,3.547",0.0,None,Gasoline,NaN,NaN,NaN,NaN,NaN


In [5]:
eventsSF.tollPaid.unique()

array([nan,  0.,  5.])

In [6]:
eventsSF.tollCost.unique()

array([nan,  0.,  5.])

In [7]:
eventsSF.columns

Index(['person', 'vehicle', 'time', 'type', 'mode', 'currentTourMode',
       'expectedMaximumUtility', 'availableAlternatives', 'location',
       'personalVehicleAvailable', 'length', 'tourIndex', 'legModes',
       'legVehicleIds', 'currentActivity', 'nextActivity', 'tripId', 'reason',
       'link', 'legMode', 'facility', 'actType', 'shiftStatus', 'parkingTaz',
       'chargingPointType', 'pricingModel', 'parkingType', 'locationY',
       'locationX', 'parkingZoneId', 'fuel', 'duration', 'vehicleType',
       'price', 'endY', 'endX', 'startY', 'startX', 'arrivalTime',
       'departureTime', 'primaryFuelLevel', 'secondaryFuelLevel', 'cost',
       'driver', 'links', 'numPassengers', 'primaryFuel', 'riders',
       'toStopIndex', 'fromStopIndex', 'seatingCapacity', 'tollPaid',
       'capacity', 'linkTravelTime', 'secondaryFuel', 'secondaryFuelType',
       'primaryFuelType', 'departTime', 'score', 'incentive', 'tollCost',
       'netCost'],
      dtype='object')

In [30]:
eventsSF[(eventsSF['tripId'] == 1306)|(eventsSF['person'] == 3)|(eventsSF['driver'] == '3')|(eventsSF['riders'] == 3)]

,price,shiftStatus,parkingTaz,chargingPointType,pricingModel,parkingType,locationY,locationX,parkingZoneId,fuel,duration,vehicleType,actType,vehicle,person,time,type,secondaryFuelLevel,primaryFuelLevel,score,driver,link,legMode,reason,departTime,startX,startY,endX,endY,mode,incentive,tollCost,netCost,arrivalTime,departureTime,currentTourMode,facility,riders,toStopIndex,fromStopIndex,seatingCapacity,tollPaid,capacity,linkTravelTime,secondaryFuel,secondaryFuelType,primaryFuelType,links,numPassengers,length,primaryFuel,expectedMaximumUtility,availableAlternatives,location,personalVehicleAvailable,tourIndex,legModes,legVehicleIds,currentActivity,nextActivity,tripId,cost
2122067,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,BODY-TYPE-DEFAULT,NaN,body-3,NaN,25837.0,PathTraversal,0.0,2.210000e+06,NaN,3,NaN,NaN,NaN,NaN,-121.806440,37.350304,-121.806440,37.350304,walk,NaN,NaN,NaN,25837.0,25837.0,car,NaN,3,NaN,NaN,0.0,0.0,0.0,NaN,0.0,None,Food,NaN,1.0,0.000,0.000000e+00,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2122495,NaN,NaN,641,None,FlatFee,Residential,NaN,NaN,NaN,NaN,NaN,NaN,NaN,539167,NaN,25837.0,LeavingParkingEvent,NaN,NaN,-0.0,3,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2178034,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,conv-L1-10000-to-25000-LowTech-2019,NaN,539167,NaN,26024.0,PathTraversal,0.0,3.650364e+09,NaN,3,NaN,NaN,NaN,NaN,-121.791738,37.337581,-121.819826,37.325748,car,NaN,NaN,NaN,26024.0,25837.0,car,NaN,NaN,NaN,NaN,3.0,0.0,3.0,"122.338,34.142,1.831,0.545,0.626,0.823,1.386,1...",0.0,None,Gasoline,"60587,56795,73546,73537,73533,73535,3442,17365...",0.0,3979.673,5.616126e+06,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2195808,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,conv-L1-10000-to-25000-LowTech-2019,NaN,539167,NaN,26051.0,PathTraversal,0.0,3.649706e+09,NaN,3,NaN,NaN,NaN,NaN,-121.819826,37.325748,-121.815924,37.322654,car,NaN,NaN,NaN,26051.0,26024.0,car,NaN,NaN,NaN,NaN,3.0,0.0,3.0,"3.009,26.882",0.0,None,Gasoline,"92800,92599",0.0,487.011,6.576112e+05,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2195809,NaN,NaN,584,None,FlatFee,Workplace,37.322654,-121.815924,NaN,NaN,NaN,NaN,NaN,539167,NaN,26051.0,ParkingEvent,NaN,NaN,NaN,3,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.0
2203831,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,BODY-TYPE-DEFAULT,NaN,body-3,NaN,26051.0,PathTraversal,0.0,2.210000e+06,NaN,3,NaN,NaN,NaN,NaN,-121.815924,37.322654,-121.815924,37.322654,walk,NaN,NaN,NaN,26051.0,26051.0,car,NaN,3,NaN,NaN,0.0,0.0,0.0,NaN,0.0,None,Food,NaN,1.0,0.000,0.000000e+00,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
10001396,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,BODY-TYPE-DEFAULT,NaN,body-3,NaN,38085.0,PathTraversal,0.0,2.209453e+06,NaN,3,NaN,NaN,NaN,NaN,-121.819826,37.325748,-121.815825,37.322605,walk,NaN,NaN,NaN,38085.0,38077.0,car,NaN,3,NaN,NaN,0.0,0.0,0.0,"374.624,7.945",0.0,None,Food,"92599,92563",1.0,10.328,5.473840e+02,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
10001402,NaN,NaN,584,None,FlatFee,Workplace,NaN,NaN,NaN,NaN,NaN,NaN,NaN,539167,NaN,38085.0,LeavingParkingEvent,NaN,NaN,-0.0,3,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
10093701,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,conv-L1-10000-to-25000-LowTech-2019,NaN,539167,NaN,38302.0,PathTraversal,0.0,3.643451e+09,NaN,3,NaN,NaN,NaN,NaN,-121.815825,37.322605,-121.802439,37.347703,car,NaN,NaN,NaN,38302.0,38085.0,car,NaN,NaN,NaN,NaN,3.0,0.0,3.0,"0.574,27.477,4.097,3.302,8.808,1.003,11.588,2....",0.0,None,Gasoline,"92562,92598,92798,176695,92805,64885,92810,570...",0.0,4453.263,6.255724e+06,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
10162253,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,conv-L1-10000-to-25000-LowTec

In [7]:
eventsSF['scenario'] = "baseline"
eventsSF['scenario'] = eventsSF['scenario'].astype("category")
eventsSF['year'] = 2018

In [8]:
# Rename the "mode" column
eventsSF.rename(columns={"mode":"modeBEAM"}, inplace=True) 
# Replace "Work" with "work" in the "actType" column
eventsSF["actType"].replace({"Work": "work"}, inplace=True)

In [9]:
eventsSFCopy = eventsSF.copy()

In [10]:
# deadhead = eventsSF[eventsSF.person.str.contains("Agent", na=False)].reset_index(drop=True)
# When there are no riders. Driver and person are not numbers, but include "Agent"

In [11]:
# Remove person = TransitDriver or RidehailDriver because there are no agent information in these rows
eventsSF = eventsSF[~eventsSF.person.str.contains("Agent", na=False)].reset_index(drop=True)

In [12]:
# shift column 'person' to first position
first_column = eventsSF.pop('person')
second_column = eventsSF.pop('driver')
third_column = eventsSF.pop('riders')
# insert column using insert(position,column_name,first_column) function
eventsSF.insert(0, 'person', first_column)
eventsSF.insert(1, 'driver', second_column)
eventsSF.insert(2, 'riders', third_column)

In [13]:
# Adding the IDMerged Column
eventsSF['UniqueID'] = eventsSF['person'] #make a copy of the person column
eventsSF['personID'] = np.where(eventsSF['person'].isin(eventsSF['driver']), eventsSF['person'], np.nan) 
eventsSF['driverID'] = np.where(eventsSF['driver'].isin(eventsSF['person']), eventsSF['driver'], np.nan)

In [14]:
# Merging person and driver ids in one column
eventsSF['IDMerged'] = eventsSF['personID'].combine_first(eventsSF['driverID'])
eventsSF['IDMerged'] = eventsSF['UniqueID'].combine_first(eventsSF['IDMerged'])

In [15]:
# Dropping unused columns
eventsSF = eventsSF.drop(['personID','driverID','UniqueID'], axis=1) 

In [16]:
# Shift column 'IDMerged' to first position
first_column = eventsSF.pop('IDMerged')
# Insert column using insert(position,column_name,first_column) function
eventsSF.insert(0, 'IDMerged', first_column)

In [17]:
%%time
# Split the "riders' column and replicated rows for every rider
eventsSF['riders'] = eventsSF['riders'].str.split(':')
eventsSF = eventsSF.explode('riders')

CPU times: total: 1min 31s
Wall time: 1min 31s


In [18]:
# Combine riderID with IDMerged
eventsSF['riderID'] = np.where(eventsSF['riders'].isin(eventsSF['person']), eventsSF['riders'], np.nan)
eventsSF['IDMerged'] = eventsSF['riderID'].combine_first(eventsSF['IDMerged'])

In [19]:
# Dropping unused columns
eventsSF = eventsSF.drop(['riderID'], axis=1) 

In [20]:
# Remove driver = TransitDriver or RidehailDriver for IDMerged = NAN because there are no agent information in these rows 
eventsSF = eventsSF[~((eventsSF.driver.str.contains("Agent", na=False))&(eventsSF.IDMerged.isna()))].reset_index(drop=True)

In [21]:
IDnan = eventsSF[eventsSF['IDMerged'].isna()]
IDnan.shape

(3380, 65)

In [22]:
%%time
# Filling NANs in ID related to charging events
eventsSF["chargeID"] = eventsSF.groupby('vehicle')['IDMerged'].transform(lambda x: x.ffill().bfill())

CPU times: total: 4min 27s
Wall time: 4min 27s


In [23]:
# Combining chargeID with IDMerged so no NANs anymore
eventsSF['IDMerged'] = eventsSF['chargeID'].combine_first(eventsSF['IDMerged'])

In [24]:
IDnan = eventsSF[eventsSF['IDMerged'].isna()]
IDnan.shape

(0, 66)

In [25]:
# Dropping unused columns
eventsSF = eventsSF.drop(['chargeID'], axis=1) 

In [26]:
print(eventsSF.currentTourMode.unique().tolist())

['hov2_teleportation', nan, 'car', 'car_hov2', 'walk', 'bike', 'hov3_teleportation', 'car_hov3', 'walk_transit', 'ride_hail', 'ride_hail_pooled', 'drive_transit', 'ride_hail_transit', 'bike_transit']


In [27]:
print(eventsSF.modeBEAM.unique().tolist())

['hov2_teleportation', nan, 'car', 'walk', 'car_hov2', 'bike', 'hov3_teleportation', 'car_hov3', 'walk_transit', 'ride_hail', 'ride_hail_pooled', 'drive_transit', 'ride_hail_transit', 'bus', 'tram', 'subway', 'rail', 'cable_car', 'bike_transit']


In [28]:
print(eventsSF.type.unique().tolist())

['ModeChoice', 'actend', 'departure', 'TeleportationEvent', 'arrival', 'actstart', 'PersonEntersVehicle', 'PathTraversal', 'LeavingParkingEvent', 'ParkingEvent', 'PersonCost', 'PersonLeavesVehicle', 'ReserveRideHail', 'Replanning', 'ChargingPlugInEvent', 'RefuelSessionEvent', 'ChargingPlugOutEvent']


In [29]:
%%time
# Change the IDMerged column type to numeric
eventsSF["IDMerged"] = pd.to_numeric(eventsSF.IDMerged)

CPU times: total: 16.7 s
Wall time: 16.7 s


In [30]:
# Sort by IDMerged and time columns
eventsSF = eventsSF.sort_values(['IDMerged','time']).reset_index(drop=True)

In [31]:
# We assume that the number of passengers is 1 for ride_hail_pooled
eventsSF['modeBEAM_rh'] = np.where(eventsSF.driver.str.contains("rideHailAgent", na=False), 'ride_hail' , eventsSF['modeBEAM'])

In [32]:
# Adding teleportation mode to the type = TeleportationEvent row 
eventsSF["modeBEAM_rh"] = np.where(eventsSF['type']=='TeleportationEvent', eventsSF.modeBEAM_rh.fillna(method='ffill'), eventsSF["modeBEAM_rh"])

In [33]:
eventsSF['modeBEAM_rh_pooled'] = np.where((eventsSF['type'] == 'PersonCost') & (eventsSF['modeBEAM'] == 'ride_hail_pooled'), 'ride_hail_pooled', np.nan)

In [34]:
eventsSF['modeBEAM_rh_ride_hail_transit'] = np.where((eventsSF['type'] == 'PersonCost') & (eventsSF['modeBEAM'] == 'ride_hail_transit'), 'ride_hail_transit', np.nan)

In [35]:
eventsSF['modeBEAM_rh_pooled'] = eventsSF['modeBEAM_rh_pooled'].shift(+1)

In [36]:
eventsSF['modeBEAM_rh_ride_hail_transit'] = eventsSF['modeBEAM_rh_ride_hail_transit'].shift(+1)

In [37]:
eventsSF['modeBEAM_rh'] = np.where((eventsSF['type'] == 'PathTraversal') & (eventsSF['modeBEAM'] == 'car') & (eventsSF['driver'].str.contains("rideHailAgent", na=False)) & (eventsSF['modeBEAM_rh_pooled'] != 'nan'), eventsSF['modeBEAM_rh_pooled'], eventsSF['modeBEAM_rh'])

In [38]:
# We don't know if ridehail_transit is ride_hail or ride_hail_pooled
eventsSF['modeBEAM_rh'] = np.where((eventsSF['type'] == 'PathTraversal') & (eventsSF['modeBEAM'] == 'car') & (eventsSF['driver'].str.contains("rideHailAgent", na=False)) & (eventsSF['modeBEAM_rh_ride_hail_transit'] != 'nan'), eventsSF['modeBEAM_rh_ride_hail_transit'], eventsSF['modeBEAM_rh'])

In [39]:
# Dropping the temporary columns
eventsSF = eventsSF.drop(['modeBEAM_rh_pooled'], axis=1)
eventsSF = eventsSF.drop(['modeBEAM_rh_ride_hail_transit'], axis=1)

#### Adding census blocks

In [40]:
def addGeometryIdToDataFrame(df, gdf, xcol, ycol, idColumn="geometry", df_geom='epsg:4326'):
    gdf_data = gpd.GeoDataFrame(df, geometry=gpd.points_from_xy(df[xcol], df[ycol]))
    gdf_data.crs = {'init': df_geom}
    joined = gpd.sjoin(gdf_data.to_crs('epsg:26910'), gdf.to_crs('epsg:26910'))
    gdf_data = gdf_data.merge(joined['blkgrpid'], left_index=True, right_index=True, how="left")
    gdf_data.rename(columns={'blkgrpid': idColumn}, inplace=True)
    df = pd.DataFrame(gdf_data.drop(columns='geometry'))
    df.drop(columns=[xcol, ycol], inplace=True)
    return df.loc[~df.index.duplicated(keep='first'), :]

In [41]:
BGs = gpd.read_file('C:/Shared-Work/Data/Scenario_2010_shp/641aa0d4-ce5b-4a81-9c30-8790c4ab8cfb202047-1-wkkklf.j5ouj.shp')

In [42]:
%%time
eventsSF = addGeometryIdToDataFrame(eventsSF, BGs, 'startX', 'startY', 'BlockGroupStart')

C:\Users\nazanin\.conda\envs\geo_env\lib\site-packages\pyproj\crs\crs.py:53: FutureWarning: '+init=<authority>:<code>' syntax is deprecated. '<authority>:<code>' is the preferred initialization method. When making the change, be mindful of axis order changes: https://pyproj4.github.io/pyproj/stable/gotchas.html#axis-order-changes-in-proj-6
  return _prepare_from_string(" ".join(pjargs))


CPU times: total: 59min 17s
Wall time: 59min 18s


In [43]:
%%time
eventsSF = addGeometryIdToDataFrame(eventsSF, BGs, 'endX', 'endY', 'BlockGroupEnd')

C:\Users\nazanin\.conda\envs\geo_env\lib\site-packages\pyproj\crs\crs.py:53: FutureWarning: '+init=<authority>:<code>' syntax is deprecated. '<authority>:<code>' is the preferred initialization method. When making the change, be mindful of axis order changes: https://pyproj4.github.io/pyproj/stable/gotchas.html#axis-order-changes-in-proj-6
  return _prepare_from_string(" ".join(pjargs))


CPU times: total: 59min 17s
Wall time: 59min 17s


#### Adding new columns

In [44]:
eventsSF['actEndTime'] = np.where(eventsSF['type']=='actend'
                     , eventsSF['time'], np.nan)

In [45]:
eventsSF['actStartTime'] = np.where(eventsSF['type']=='actstart'
                     , eventsSF['time'], np.nan)    

In [46]:
eventsSF['duration_travelling'] = np.where((eventsSF['type']=='PathTraversal')|(eventsSF['type']=='TeleportationEvent')
                     , eventsSF['arrivalTime'] - eventsSF['departureTime'], np.nan)

In [47]:
eventsSF['distance_travelling'] = np.where((eventsSF['type']=='PathTraversal')|((eventsSF['type']=='ModeChoice')&((eventsSF['modeBEAM']=='hov2_teleportation')|(eventsSF['modeBEAM']=='hov3_teleportation'))), eventsSF['length'], np.nan)

In [48]:
eventsSF['distance_mode_choice'] = np.where(eventsSF['type']=='ModeChoice', eventsSF['length'], np.nan)

In [49]:
eventsSF['duration_walking'] = np.where(eventsSF['modeBEAM']=='walk', eventsSF['duration_travelling'], np.nan)

In [50]:
eventsSF['distance_walking'] = np.where(eventsSF['modeBEAM']=='walk', eventsSF['distance_travelling'], np.nan)

In [51]:
eventsSF['duration_on_bike'] = np.where(eventsSF['modeBEAM']=='bike', eventsSF['duration_travelling'], np.nan)

In [52]:
eventsSF['distance_bike'] = np.where(eventsSF['modeBEAM']=='bike', eventsSF['distance_travelling'], np.nan)

In [53]:
eventsSF['duration_in_ridehail'] = np.where((eventsSF['modeBEAM_rh']=='ride_hail')|(eventsSF['modeBEAM_rh']=='ride_hail_pooled')|(eventsSF['modeBEAM_rh']=='ride_hail_transit'), 
                                            eventsSF['duration_travelling'], np.nan)

In [54]:
eventsSF['distance_ridehail'] = np.where((eventsSF['modeBEAM_rh']=='ride_hail')|(eventsSF['modeBEAM_rh']=='ride_hail_pooled')|(eventsSF['modeBEAM_rh']=='ride_hail_transit'), eventsSF['distance_travelling'], np.nan)

In [55]:
eventsSF['duration_in_privateCar'] = np.where((eventsSF['modeBEAM_rh']=='car')|(eventsSF['modeBEAM_rh']=='car_hov3')|(eventsSF['modeBEAM_rh']=='car_hov2')|
                                              (eventsSF['modeBEAM_rh']=='hov2_teleportation')|(eventsSF['modeBEAM_rh']=='hov3_teleportation') 
                                              , eventsSF['duration_travelling'], np.nan)

In [56]:
eventsSF['distance_privateCar'] = np.where((eventsSF['modeBEAM_rh']=='car')|(eventsSF['modeBEAM_rh']=='car_hov3')|(eventsSF['modeBEAM_rh']=='car_hov2')|
                                              (eventsSF['modeBEAM_rh']=='hov2_teleportation')|(eventsSF['modeBEAM_rh']=='hov3_teleportation'), eventsSF['distance_travelling'], np.nan)

In [57]:
eventsSF['duration_in_transit'] = np.where((eventsSF['modeBEAM']=='bike_transit')|(eventsSF['modeBEAM']=='drive_transit')|
                                           (eventsSF['modeBEAM']=='walk_transit')|(eventsSF['modeBEAM']=='bus')|
                                           (eventsSF['modeBEAM']=='tram')|(eventsSF['modeBEAM']=='subway')|
                                           (eventsSF['modeBEAM']=='rail')|(eventsSF['modeBEAM']=='cable_car')|
                                           (eventsSF['modeBEAM']=='ride_hail_transit'), eventsSF['duration_travelling'], np.nan)

In [58]:
eventsSF['distance_transit'] = np.where((eventsSF['modeBEAM']=='bike_transit')|(eventsSF['modeBEAM']=='drive_transit')|
                                        (eventsSF['modeBEAM']=='walk_transit')|(eventsSF['modeBEAM']=='bus')|
                                        (eventsSF['modeBEAM']=='tram')|(eventsSF['modeBEAM']=='subway')|
                                        (eventsSF['modeBEAM']=='rail')|(eventsSF['modeBEAM']=='cable_car')|
                                        (eventsSF['modeBEAM']=='ride_hail_transit'), eventsSF['distance_travelling'], np.nan)

In [59]:
# Removing the extra tour index happening after replanning events
eventsSF['replanningTime'] = np.where(eventsSF['type'] == 'Replanning', eventsSF['time'], np.nan)
eventsSF['replanningTime'] = eventsSF['replanningTime'].shift(+1)
eventsSF['tourIndex_fixed'] = np.where((eventsSF['type'] == 'ModeChoice')&(eventsSF['replanningTime'].notna()), np.nan, eventsSF['tourIndex'])

In [60]:
#eventsSF = eventsSF.set_index('IDMerged')

In [61]:
eventsSF.modeBEAM_rh.unique()

array(['hov3_teleportation', nan, 'car', 'walk', 'ride_hail', 'car_hov2',
       'walk_transit', 'bus', 'subway', 'hov2_teleportation', 'rail',
       'bike', 'car_hov3', 'tram', 'cable_car', 'drive_transit',
       'ride_hail_pooled', 'ride_hail_transit', 'bike_transit'],
      dtype=object)

In [62]:
eventsSF.primaryFuelType.unique()

[NaN, 'Food', 'Gasoline', 'Diesel', 'Electricity', 'Biodiesel']
Categories (5, object): ['Gasoline', 'Biodiesel', 'Diesel', 'Electricity', 'Food']

In [63]:
eventsSF['fuelFood'] = np.where((eventsSF['type']=='PathTraversal')&(eventsSF['primaryFuelType']=='Food'), 
                                eventsSF['primaryFuel'], np.nan)

In [64]:
eventsSF['emissionFood'] = eventsSF['fuelFood'] * 8.3141841e-9 * 0

In [65]:
eventsSF['fuelElectricity'] = np.where((eventsSF['type']=='PathTraversal')&(eventsSF['primaryFuelType']=='Electricity'), 
                                eventsSF['primaryFuel'], np.nan)

In [66]:
eventsSF['emissionElectricity'] = eventsSF['fuelElectricity'] * 2.77778e-10 * 947.2 * 0.0005

In [67]:
eventsSF['fuelDiesel'] = np.where((eventsSF['type']=='PathTraversal')&(eventsSF['primaryFuelType']=='Diesel'), 
                                eventsSF['primaryFuel'], np.nan)

In [68]:
eventsSF['emissionDiesel'] = eventsSF['fuelDiesel'] * 8.3141841e-9 * 10.180e-3

In [69]:
eventsSF['fuelBiodiesel'] = np.where((eventsSF['type']=='PathTraversal')&(eventsSF['primaryFuelType']=='Biodiesel'), 
                                eventsSF['primaryFuel'], np.nan)

In [70]:
eventsSF['emissionBiodiesel'] = eventsSF['fuelBiodiesel'] * 8.3141841e-9 * 10.180e-3

In [71]:
eventsSF['fuel_not_Food'] = np.where((eventsSF['type']=='PathTraversal')&(eventsSF['primaryFuelType']!='Food')
                            , eventsSF['primaryFuel']+eventsSF['secondaryFuel'], np.nan)

In [72]:
eventsSF['fuelGasoline'] = np.where((eventsSF['type']=='PathTraversal')&((eventsSF['primaryFuelType']=='Gasoline')|(eventsSF['secondaryFuelType']=='Gasoline')), 
                           eventsSF['primaryFuel']+eventsSF['secondaryFuel'], np.nan)

In [73]:
eventsSF['emissionGasoline'] = eventsSF['fuelGasoline'] * 8.3141841e-9 * 8.89e-3

In [74]:
# Marginal fuel
conditions  = [(eventsSF['modeBEAM_rh'] == 'ride_hail_pooled'), 
               (eventsSF['modeBEAM_rh'] == 'walk_transit') | (eventsSF['modeBEAM_rh'] == 'drive_transit')|
               (eventsSF['modeBEAM_rh'] == 'ride_hail_transit')|(eventsSF['modeBEAM_rh'] == 'bus')|(eventsSF['modeBEAM_rh'] == 'subway')|
               (eventsSF['modeBEAM_rh'] == 'rail')|(eventsSF['modeBEAM_rh'] == 'tram')|(eventsSF['modeBEAM_rh'] == 'cable_car')|
               (eventsSF['modeBEAM_rh'] == 'bike_transit'),
               (eventsSF['modeBEAM_rh'] == 'walk')|(eventsSF['modeBEAM_rh'] == 'bike'),
               (eventsSF['modeBEAM_rh'] == 'ride_hail')|(eventsSF['modeBEAM_rh'] == 'car')| 
               (eventsSF['modeBEAM_rh'] == 'car_hov2')| (eventsSF['modeBEAM_rh'] == 'car_hov3')|
               (eventsSF['modeBEAM_rh'] == 'hov2_teleportation')| (eventsSF['modeBEAM_rh'] == 'hov3_teleportation')]
choices = [eventsSF['fuel_not_Food']/eventsSF['numPassengers'], 0 , eventsSF['fuelFood'], eventsSF['fuel_not_Food']]

In [75]:
eventsSF['fuel_marginal'] = np.select(conditions, choices, default=np.nan)

In [76]:
#eventsSF[(eventsSF['modeBEAM_rh'] == 'ride_hail_pooled') & (eventsSF['fuelElectricity'].notna() != 0)].head()

In [77]:
# Marginal emission
conditions1  = [(eventsSF['modeBEAM_rh'] == 'ride_hail_pooled') & (eventsSF['fuelElectricity'].notna() != 0), 
               (eventsSF['modeBEAM_rh'] == 'ride_hail_pooled') & (eventsSF['fuelGasoline'].notna() != 0),
               (eventsSF['modeBEAM_rh'] == 'ride_hail_pooled') & (eventsSF['fuelBiodiesel'].notna() != 0),
               (eventsSF['modeBEAM_rh'] == 'ride_hail_pooled') & (eventsSF['fuelDiesel'].notna() != 0),             
               (eventsSF['modeBEAM_rh'] == 'walk_transit') | (eventsSF['modeBEAM_rh'] == 'drive_transit')|
               (eventsSF['modeBEAM_rh'] == 'ride_hail_transit')|(eventsSF['modeBEAM_rh'] == 'bus')|(eventsSF['modeBEAM_rh'] == 'subway')|
               (eventsSF['modeBEAM_rh'] == 'rail')|(eventsSF['modeBEAM_rh'] == 'tram')|(eventsSF['modeBEAM_rh'] == 'cable_car')|
               (eventsSF['modeBEAM_rh'] == 'bike_transit'),

               (eventsSF['modeBEAM_rh'] == 'walk')|(eventsSF['modeBEAM_rh'] == 'bike'),
               
               (eventsSF['modeBEAM_rh'] == 'ride_hail')|(eventsSF['modeBEAM_rh'] == 'car')| 
               (eventsSF['modeBEAM_rh'] == 'car_hov2')| (eventsSF['modeBEAM_rh'] == 'car_hov3')|
               (eventsSF['modeBEAM_rh'] == 'hov2_teleportation')| (eventsSF['modeBEAM_rh'] == 'hov3_teleportation')&
               (eventsSF['fuelElectricity'].notna() != 0),
              
               (eventsSF['modeBEAM_rh'] == 'ride_hail')|(eventsSF['modeBEAM_rh'] == 'car')| 
               (eventsSF['modeBEAM_rh'] == 'car_hov2')| (eventsSF['modeBEAM_rh'] == 'car_hov3')|
               (eventsSF['modeBEAM_rh'] == 'hov2_teleportation')| (eventsSF['modeBEAM_rh'] == 'hov3_teleportation')&
               (eventsSF['fuelGasoline'].notna() != 0),           
              
               (eventsSF['modeBEAM_rh'] == 'ride_hail')|(eventsSF['modeBEAM_rh'] == 'car')| 
               (eventsSF['modeBEAM_rh'] == 'car_hov2')| (eventsSF['modeBEAM_rh'] == 'car_hov3')|
               (eventsSF['modeBEAM_rh'] == 'hov2_teleportation')| (eventsSF['modeBEAM_rh'] == 'hov3_teleportation')&
               (eventsSF['fuelBiodiesel'].notna() != 0),   
               
               (eventsSF['modeBEAM_rh'] == 'ride_hail')|(eventsSF['modeBEAM_rh'] == 'car')| 
               (eventsSF['modeBEAM_rh'] == 'car_hov2')| (eventsSF['modeBEAM_rh'] == 'car_hov3')|
               (eventsSF['modeBEAM_rh'] == 'hov2_teleportation')| (eventsSF['modeBEAM_rh'] == 'hov3_teleportation')&
               (eventsSF['fuelDiesel'].notna() != 0),

               (eventsSF['modeBEAM_rh'] == 'ride_hail')|(eventsSF['modeBEAM_rh'] == 'car')| 
               (eventsSF['modeBEAM_rh'] == 'car_hov2')| (eventsSF['modeBEAM_rh'] == 'car_hov3')|
               (eventsSF['modeBEAM_rh'] == 'hov2_teleportation')| (eventsSF['modeBEAM_rh'] == 'hov3_teleportation')&
               (eventsSF['fuelFood'].notna() != 0)]

choices1 = [eventsSF['emissionElectricity']/eventsSF['numPassengers'],
           eventsSF['emissionGasoline']/eventsSF['numPassengers'],
           eventsSF['emissionBiodiesel']/eventsSF['numPassengers'],
           eventsSF['emissionDiesel']/eventsSF['numPassengers'],           
           0 , 
           eventsSF['emissionFood'], 
           eventsSF['emissionElectricity'],
           eventsSF['emissionGasoline'],
           eventsSF['emissionBiodiesel'],
           eventsSF['emissionDiesel'],
           eventsSF['emissionFood']]

In [78]:
eventsSF['emission_marginal'] = np.select(conditions1, choices1, default=np.nan)

In [79]:
eventsSF['actEndType'] = np.where(eventsSF['type']=='actend', eventsSF['actType'], "")

In [80]:
eventsSF['actStartType'] = np.where(eventsSF['type']=='actstart', eventsSF['actType'], "")

#### Trip Index

In [81]:
#eventsSF["tripIndex"] = eventsSF.groupby("IDMerged")["tourIndex_fixed"].rank(method="first", ascending=True)
eventsSF["tripIndex"] = eventsSF.tripId.fillna(method='ffill')

#### Previous code for mode choice 

In [82]:
# Make a new column which determined mode choice numbers for each person and trip
#s = eventsSF.groupby(['IDMerged','tripIndex', 'type']).cumcount().add(1).astype(str).str.zfill(2)

In [83]:
#eventsSF['type_number'] = eventsSF['type'].astype(str) + s.astype(str)

In [84]:
#eventsSF['mode_choice_planned_BEAM'] = np.where(eventsSF['type_number'] == 'ModeChoice01' , eventsSF['modeBEAM_rh'], np.nan)

In [85]:
# Just showing the mode choice after replanning
#eventsSF['mode_choice_replanned_BEAM'] = np.where(eventsSF['type_number'] == 'ModeChoice02', eventsSF['modeBEAM_rh'], np.nan)

#### Only works if we have one replanning event

In [ ]:
# Showing both mode choices before and after the replanning event
#eventsSF['mode_choice_act'] = eventsSF['mode_choice_replanned_BEAM'].combine_first(eventsSF['mode_choice_planned_BEAM'])

In [ ]:
# If replanned, then the status is 1
#eventsSF['replanning_status'] = np.where(eventsSF['mode_choice_replanned_BEAM'].notna() , 1, np.nan)

In [ ]:
# Group by and fill all the rows related to the replanning event
#eventsSF["replanning_status"] = eventsSF.groupby(['IDMerged','tripIndex'], as_index=False)['replanning_status'].transform(lambda x: x.ffill().bfill())

In [ ]:
# Put all non-replanning events equal to 0
#eventsSF["replanning_status"] = np.where(eventsSF['replanning_status'].isna(), 0, eventsSF['replanning_status']).astype(int)

In [ ]:
#eventsSF['mode_choice_actual_BEAM'] = np.where((eventsSF['replanning_status'] == 1) & (eventsSF['type_number']=='ModeChoice01'), np.nan, eventsSF['mode_choice_act'])

In [ ]:
#eventsSF = eventsSF.drop(['mode_choice_act'], axis = 1)

In [ ]:
#eventsSF[eventsSF['type'] == "Replanning"].tail(16)

In [ ]:
#num_pass.to_csv('C:/Shared-Work/Data/CleanData/num_pass.csv', index = False)

In [ ]:
#eventsSF["mode_choice_replanned_BEAM"] = eventsSF.groupby(['IDMerged','tripIndex'])['mode_choice_replanned_BEAM'].transform(lambda x: x.bfill())

In [ ]:
#eventsSF['mode_choice_actual_BEAM'] = eventsSF['mode_choice_actual_BEAM'].combine_first(eventsSF['modeBEAM_rh'])

In [ ]:
#eventsSF['mode_planned_tour_ActSim'] = np.where(eventsSF['type'] == 'PathTraversal', eventsSF['currentTourMode'], np.nan)

In [ ]:
#eventsSF.head(10000).to_csv('C:/Shared-Work/Data/CleanData/eventsNew_check.csv', index = False)   

In [ ]:
#eventsSF['numPassengers_agg'] = eventsSF.groupby(['IDMerged','tripIndex'], as_index=False).numPassengers.agg(lambda x: ', '.join(set(x.dropna().astype(str))))

#### Mode Choice planned and actual

In [86]:
%%time
eventsSF['mode_choice_actual_BEAM'] = eventsSF.groupby(['IDMerged','tripId', 'type'])['modeBEAM'].transform('last')

CPU times: total: 1min 27s
Wall time: 1min 27s


In [87]:
%%time
eventsSF['mode_choice_planned_BEAM'] = eventsSF.groupby(['IDMerged','tripId', 'type'])['modeBEAM'].transform('first')

CPU times: total: 1min 26s
Wall time: 1min 26s


In [88]:
eventsSF['mode_choice_actual_BEAM'] = np.where(eventsSF['type'] != 'ModeChoice' , np.nan, eventsSF['mode_choice_actual_BEAM'])

In [89]:
eventsSF['mode_choice_planned_BEAM'] = np.where(eventsSF['type'] != 'ModeChoice' , np.nan, eventsSF['mode_choice_planned_BEAM'])

In [90]:
# Rename the "netCost" column
eventsSF.rename(columns={"netCost":"cost_BEAM"}, inplace=True) 

In [91]:
#eventsSF[eventsSF['type']=='Replanning'].head()

In [92]:
# Replanning events = 1, the rest = 0
eventsSF['replanning_status'] = np.where(eventsSF['type']=='Replanning', 1, 0)

In [93]:
#q = eventsSF[eventsSF['IDMerged'] == 6902751]

In [94]:
#q.to_csv('C:/Shared-Work/Data/CleanData/PivotTables/fake2.csv')   

In [95]:
%%time
eventsSF['reason'].replace('nan', np.NaN)

CPU times: total: 1.67 s
Wall time: 1.67 s


In [96]:
eventsSF['transit_bus'] = np.where(eventsSF['modeBEAM_rh']=='bus', 1, 0)
eventsSF['transit_subway'] = np.where(eventsSF['modeBEAM_rh']=='subway', 1, 0)
eventsSF['transit_tram'] = np.where(eventsSF['modeBEAM_rh']=='tram', 1, 0)
eventsSF['transit_rail'] = np.where(eventsSF['modeBEAM_rh']=='rail', 1, 0)
eventsSF['transit_cable_car'] = np.where(eventsSF['modeBEAM_rh']=='cable_car', 1, 0)

In [97]:
eventsSF['ride_hail_pooled'] = np.where(eventsSF['modeBEAM_rh']=='ride_hail_pooled', 1, 0)

In [98]:
#transit_data = eventsSF[(eventsSF['modeBEAM_rh'] == "bus") | (eventsSF['modeBEAM_rh'] == "subway") 
#                        | (eventsSF['modeBEAM_rh'] == 'walk_transit') | (eventsSF['modeBEAM_rh'] == 'ride_hail_transit') 
#                        | (eventsSF['modeBEAM_rh'] == 'bike_transit') | (eventsSF['modeBEAM_rh'] == 'drive_transit') 
#                        | (eventsSF['modeBEAM_rh'] == 'cable_car') | (eventsSF['modeBEAM_rh'] == 'rail') 
#                        | (eventsSF['modeBEAM_rh'] == 'tram')]

In [99]:
#eventsSF.loc['Total',:]= df.sum(axis=0)

In [ ]:
#eventsSF.type[eventsSF.type == 'Replanning'].count()

In [ ]:
#Person_Trip_eventsSF.replanning_status[Person_Trip_eventsSF.replanning_status == 0].count()

In [ ]:
#rh_transit5 = eventsSF[eventsSF['IDMerged']== 3671596]

In [ ]:
#rh_transit5.to_csv('C:/Shared-Work/Data/CleanData/rh_transit5.csv', index = False)

In [100]:
eventsSFCopy = eventsSF.copy()

In [101]:
eventsSF.head(200000).to_csv('C:/Shared-Work/Data/CleanData/modeBEAMCHECK.csv', index = False)

In [102]:
%%time
Person_Trip_eventsSF = pd.pivot_table(
   eventsSF,
   index=['IDMerged','tripIndex'],
   aggfunc={'actStartTime': np.sum, 'actEndTime': np.sum, 'duration_travelling': np.sum, 'cost_BEAM': np.sum, 'actStartType': np.sum, 
            'actEndType': np.sum, 'duration_walking': np.sum, 'duration_in_privateCar': np.sum, 'duration_on_bike': np.sum, 
            'duration_in_ridehail': np.sum, 'distance_travelling': np.sum, 'duration_in_transit': np.sum, 'distance_walking': np.sum, 
            'distance_bike': np.sum, 'distance_ridehail': np.sum, 'distance_privateCar': np.sum, 'distance_transit': np.sum, 
            'legVehicleIds': np.sum, 
            'mode_choice_planned_BEAM':lambda x: ', '.join(set(x.dropna().astype(str))),
            'mode_choice_actual_BEAM':lambda x: ', '.join(set(x.dropna().astype(str))),
            'vehicle': lambda x: ', '.join(set(x.dropna().astype(str))),
            'numPassengers': lambda x: ', '.join(list(x.dropna().astype(str))),
            'distance_mode_choice': np.sum,
            'replanning_status': np.sum,
            'reason': lambda x: ', '.join(list(x.dropna().astype(str))),
            'parkingType': lambda x: ', '.join(list(x.dropna().astype(str))),
            'transit_bus': np.sum, 'transit_subway': np.sum, 'transit_tram': np.sum, 'transit_cable_car': np.sum,
            'ride_hail_pooled': np.sum, 'transit_rail': np.sum,
            'year': lambda x: ', '.join(set(x.dropna().astype(str))),
            'scenario': lambda x: ', '.join(set(x.dropna().astype(str))),
            'fuelFood': np.sum, 'fuelElectricity': np.sum, 'fuelBiodiesel': np.sum, 
            'fuelDiesel': np.sum, 'fuel_not_Food': np.sum, 'fuelGasoline': np.sum, 'fuel_marginal': np.sum,
            'BlockGroupStart': 'first',
            'BlockGroupEnd': 'last',
            'emissionFood': np.sum, 'emissionElectricity': np.sum, 'emissionDiesel': np.sum, 'emissionGasoline': np.sum,
            'emissionBiodiesel': np.sum, 'emission_marginal': np.sum
           }).reset_index() 

#'numPassengers': lambda x: ', '.join(set(x.dropna().astype(str))) 
#'mode_choice_actual_BEAM':lambda x: ', '.join(set(x.dropna().astype(str))) #
#'modeBEAM_rh': lambda x: ', '.join(list(x.dropna().astype(str))), 

CPU times: total: 38min 44s
Wall time: 38min 44s


In [103]:
Person_Trip_eventsSF['duration_door_to_door'] = Person_Trip_eventsSF['actStartTime'] - Person_Trip_eventsSF['actEndTime'] 

In [104]:
Person_Trip_eventsSF['waitTime'] = Person_Trip_eventsSF['duration_door_to_door'] - Person_Trip_eventsSF['duration_travelling'] 

In [105]:
Person_Trip_eventsSF['actPurpose'] = Person_Trip_eventsSF['actEndType'].astype(str) + "_to_" + Person_Trip_eventsSF['actStartType'].astype(str)

In [ ]:
#Person_Trip_eventsSF[Person_Trip_eventsSF['IDMerged'] == 2192]

In [ ]:
#Person_Trip_eventsSF.rename(columns={"modeBEAM_rh":"mode_BEAM_list"}, inplace=True) 

In [106]:
Person_Trip_eventsSF.rename(columns={"legVehicleIds":"vehicleIds_estimate"}, inplace=True) 

In [107]:
Person_Trip_eventsSF.rename(columns={"vehicle":"vehicleIds"}, inplace=True) 

In [108]:
#Person_Trip_eventsSF.door_to_door_time[Person_Trip_eventsSF.door_to_door_time<0].count()

In [109]:
#negative_door = Person_Trip_eventsSF[(Person_Trip_eventsSF.door_to_door_time<0)]

In [110]:
# Column with five summarized modes
conditions  = [(Person_Trip_eventsSF['mode_choice_actual_BEAM'] == 'ride_hail') | (Person_Trip_eventsSF['mode_choice_actual_BEAM'] == 'ride_hail_pooled'), 
               (Person_Trip_eventsSF['mode_choice_actual_BEAM'] == 'walk_transit') | (Person_Trip_eventsSF['mode_choice_actual_BEAM'] == 'drive_transit')| (Person_Trip_eventsSF['mode_choice_actual_BEAM'] == 'ride_hail_transit')|(Person_Trip_eventsSF['mode_choice_actual_BEAM'] == 'bike_transit'),
               (Person_Trip_eventsSF['mode_choice_actual_BEAM'] == 'walk'), (Person_Trip_eventsSF['mode_choice_actual_BEAM'] == 'bike'),
               (Person_Trip_eventsSF['mode_choice_actual_BEAM'] == 'car') | (Person_Trip_eventsSF['mode_choice_actual_BEAM'] == 'car_hov2')| (Person_Trip_eventsSF['mode_choice_actual_BEAM'] == 'car_hov3')|(Person_Trip_eventsSF['mode_choice_actual_BEAM'] == 'hov2_teleportation')| (Person_Trip_eventsSF['mode_choice_actual_BEAM'] == 'hov3_teleportation')]
choices = [ 'ride_hail', 'transit', 'walk', 'bike', 'car']

In [111]:
Person_Trip_eventsSF['mode_choice_actual_5'] = np.select(conditions, choices, default=np.nan)

In [112]:
#negative_door.to_csv('C:/Shared-Work/Data/CleanData/PivotTables/negative_door.csv')   

In [113]:
Person_Trip_eventsSF = Person_Trip_eventsSF.drop(Person_Trip_eventsSF[Person_Trip_eventsSF.duration_door_to_door < 0].index)

In [114]:
Person_Trip_eventsSF.to_csv('C:/Shared-Work/Data/CleanData/sf_2018_base_core_act.csv', index = False)

In [ ]:
#Person_Trip_eventsSF[Person_Trip_eventsSF['mode_choice_planned_BEAM'] == 'walk_transit'].head()

#### Merging with activity sim persons and housholds files

In [7]:
actloc_2018_baseline = "https://beam-outputs.s3.amazonaws.com/pilates-outputs/sfbay-base-20220409/activitysim/"
actloc_2018_transitFreq05 = "https://beam-outputs.s3.amazonaws.com/pilates-outputs/sfbay-transit_frequencies_0.5-20220528/activitysim/"
actloc_2018_transitFreq1p5 = "https://beam-outputs.s3.amazonaws.com/pilates-outputs/sfbay-transit_frequencies_1.5-20220529/activitysim/"
actloc_2018_transitFreq2 = "https://beam-outputs.s3.amazonaws.com/pilates-outputs/sfbay-transit_frequencies_2.0-20220529/activitysim/"
actloc_2018_rhFleetsz0125 = "https://beam-outputs.s3.amazonaws.com/pilates-outputs/sfbay-RH_fleetsz_0.125-20220408/activitysim/"
actloc_2018_rhFleetsz175 = "https://beam-outputs.s3.amazonaws.com/pilates-outputs/sfbay-RH_fleetsz_1.75-20220408/activitysim/"

households = pd.read_csv(actloc_2018_transitFreq05 + 'final_households.csv')
persons = pd.read_csv(actloc_2018_transitFreq05 + 'final_persons.csv')
tours = pd.read_csv(actloc_2018_transitFreq05 +'final_tours.csv')
plans = pd.read_csv(actloc_2018_transitFreq05 +'final_plans.csv')
trips = pd.read_csv(actloc_2018_transitFreq05 + 'final_trips.csv')
landuse = pd.read_csv(actloc_2018_transitFreq05 + 'final_land_use.csv')

In [8]:
# PUMS data for Disability information
filename = "C:/Shared-Work/Data/Disability_PUMS/custom_mpo_06197001_model_data.h5"

with h5py.File(filename, "r") as f:
    # List all groups
    print("Keys: %s" % f.keys())
    a_group_key = list(f.keys())[0]

    # Get the data
    data = list(f[a_group_key])

Keys: <KeysViewHDF5 ['blocks', 'counties', 'ect', 'edges', 'hct', 'household_targets_acs', 'household_validation_acs', 'households', 'job_county_targets_BEA', 'job_county_validation_BEA', 'job_flows', 'job_targets', 'job_validation', 'jobs', 'nodes', 'persons', 'price_targets', 'price_validation', 'residential_units', 'travel_data', 'unit_county_targets_PEP', 'unit_county_validation_PEP', 'unit_targets', 'unit_validation', 'validation_ect', 'validation_hct', 'values']>


In [9]:
# PUMS data for Disability information
person_5_year_2010 = pd.read_csv('C:/Users/nazanin/Downloads/csv_pca/ss13pca.csv')
housing_unit_5_year_2010 = pd.read_csv('C:/Users/nazanin/Downloads/csv_hca/ss13hca.csv')

C:\Users\nazanin\AppData\Local\Temp\2\ipykernel_5932\1998398585.py:2: DtypeWarning: Columns (107,108,109) have mixed types. Specify dtype option on import or set low_memory=False.
  person_5_year_2010 = pd.read_csv('C:/Users/nazanin/Downloads/csv_pca/ss13pca.csv')


In [10]:
# Merge PUMS households and persons 
person_5_year_2010 = person_5_year_2010.sort_values(by=['SERIALNO']).reset_index()
housing_unit_5_year_2010 = housing_unit_5_year_2010.sort_values(by=['SERIALNO']).reset_index()
hhpersons_PUMS2010 = pd.merge(left=person_5_year_2010, right=housing_unit_5_year_2010, how='left', on='SERIALNO')

In [11]:
# Merge BEAM households and persons 
persons = persons.sort_values(by=['household_id']).reset_index(drop=True)
households = households.sort_values(by=['household_id']).reset_index(drop=True)
hhpersons = pd.merge(left=persons, right=households, how='left', on='household_id')
#hhpersons = pd.merge(left=persons, right=households, how='left', on='household_id', suffixes=('', '_drop'))
#hhpersons.drop([col for col in hhpersons.columns if 'drop' in col], axis=1, inplace=True)

In [12]:
# Merge Disability Columns
hhpersons_PUMS2010 = hhpersons_PUMS2010.sort_values(by=['SERIALNO', 'SPORDER']).reset_index(drop=True)
hhpersons = hhpersons.sort_values(by=['serialno', 'PNUM']).reset_index(drop=True)
hhpersonsDIS = pd.merge(hhpersons, hhpersons_PUMS2010[['SERIALNO', 'SPORDER', 'AGEP', 'SEX', 'DIS', 'HINCP', 'VEH', 'JWMNP', 'JWRIP', 'JWTR', 'RAC1P', 'RAC2P05']], how='left',left_on = ['serialno', 'PNUM'] , right_on=['SERIALNO', 'SPORDER'])

In [13]:
#hhpersons[hhpersons['serialno']==2009000000022]

In [14]:
# Merge tours, households and persons
tours = tours.sort_values(by=['person_id']).reset_index(drop=True)
hhpersonsDIS = hhpersonsDIS.sort_values(by=['person_id']).reset_index(drop=True)
hhperTours = pd.merge(left=tours, right=hhpersonsDIS, how='left', on='person_id')
#hhperTours = pd.merge(left=tours, right=hhpersons, how='left', on='person_id', suffixes=('', '_drop'))
#hhperTours.drop([col for col in hhperTours.columns if 'drop' in col], axis=1, inplace=True)

In [15]:
# Merge trips, tours, households and persons
trips = trips.sort_values(by=['person_id', 'tour_id']).reset_index(drop=True)
hhperTours = hhperTours.sort_values(by=['person_id','tour_id']).reset_index(drop=True)
tourTripsMerged = pd.merge(left=trips, right=hhperTours, how='left', on=['person_id','tour_id'])
#tourTripsMerged = pd.merge(left=trips, right=hhperTours, how='left', on=['person_id','tour_id'], suffixes=('', '_drop'))
#tourTripsMerged.drop([col for col in tourTripsMerged.columns if 'drop' in col], axis=1, inplace=True)

In [16]:
tourTripsMerged.head()

,trip_id,person_id,household_id,tour_id,primary_purpose_x,trip_num,outbound,trip_count,purpose,destination_x,origin_x,destination_logsum_x,depart,trip_mode,mode_choice_logsum_x,tour_type,tour_type_count,tour_type_num,tour_num,tour_count,tour_category,number_of_participants,destination_y,origin_y,household_id_x,tdd,start,end,duration,composition,destination_logsum_y,tour_mode,mode_choice_logsum_y,atwork_subtour_frequency,parent_tour_id,stop_frequency,primary_purpose_y,earning,worker,student,hispanic.1,person_sex,PNUM,sex,household_id_y,hours,MAR,person_age,edu,school_zone_id,race_id,race,work_at_home,age,work_zone_id,p_hispanic,relate,hispanic,TAZ_x,ptype,pemploy,pstudent,home_x,home_y,age_16_to_19,age_16_p,adult,male,female,has_non_worker,has_retiree,has_preschool_kid,has_driving_kid,has_school_kid,has_full_time,has_part_time,has_university,student_is_employed,nonstudent_to_school,is_student,is_gradeschool,is_highschool,is_university,school_segment,is_worker,home_taz,value_of_time,school_taz,distance_to_school,roundtrip_auto_time_to_school,workplace_taz,workplace_location_logsum,distance_to_work,workplace_in_cbd,work_taz_area_type,roundtrip_auto_time_to_work,work_auto_savings,work_auto_savings_ratio,free_parking_at_work,cdap_activity,travel_active,under16_not_at_school,has_preschool_kid_at_home,has_school_kid_at_home,mandatory_tour_frequency,work_and_school_and_worker,work_and_school_and_student,num_mand,num_work_tours,num_joint_tours,non_mandatory_tour_frequency,num_non_mand,num_escort_tours,num_eatout_tours,num_shop_tours,num_maint_tours,num_discr_tours,num_social_tours,num_non_escort_tours,gt55,seniors,VEHICL,lcm_county_id,hh_children,block_id,gt2,hispanic_head,age_of_head,race_of_head,tenure_mover,hh_size,sf_detached,tenure,hh_cars,income,hh_age_of_head,serialno,num_workers,hh_race_of_head,hh_income,recent_mover,hh_workers,hispanic_status_of_head,hh_seniors,hhsize,hh_type,TAZ_y,HHT,sample_rate,chunk_id,income_in_thousands,income_segment,median_value_of_time,hh_value_of_time,num_non_workers,num_drivers,num_adults,num_children,num_young_children,num_children_5_to_15,num_children_16_to_17,num_college_age,num_young_adults,non_family,family,home_is_urban,home_is_rural,auto_ownership,hh_work_auto_savings_ratio,num_under16_not_at_school,num_travel_active,num_travel_active_adults,num_travel_active_preschoolers,num_travel_active_children,num_travel_active_non_preschoolers,participates_in_jtf_model,joint_tour_frequency,num_hh_joint_tours,SERIALNO,SPORDER,AGEP,SEX,DIS,HINCP,VEH,JWMNP,JWRIP,JWTR,RAC1P,RAC2P05
0,577,1,1735309,72,school,1,True,2,othmaint,583,641,6.874664,8.0,SHARED3FREE,-1.977407,school,1,1,1,1,mandatory,1,636.0,641.0,1735309,61.0,8.0,15.0,7.0,NaN,NaN,SHARED3FREE,-0.969775,NaN,NaN,1out_0in,school,0.0,0,1,0,male,3,1,1735309,0.0,5,19 and under,1.0,633,6,asian,0,3,-1,no,2,0,641,8,4,1,-121.80644,37.350304,False,False,False,True,False,False,False,True,False,False,True,False,False,False,False,True,True,False,False,1,False,641,19.798008,636,1.826168,29.766666,-1,NaN,NaN,False,NaN,0.00,0.0000,0.0,False,M,True,False,False,False,school1,False,False,1,0,0,0,0,0,0,0,0,0,0,0,0,0.0,2,6085,yes,60855035091006,1,no,35,6,own not recent,four or more,yes,1,two or more,240000.0,gt35-lt65,2010000482865,2.0,asian,gt150,0,two or more,0,no,4,3,641,4,0.097,60138,240.0,4,12.86,29.682171,2.0,2,2,2,2,0,0,0,1,True,False,False,False,2,1.245161,0,4,2,2,2,2,True,0_tours,0,2010000482865,3,3,1,2,240000.0,2.0,NaN,NaN,NaN,6,40
1,578,1,1735309,72,school,2,True,2,school,636,583,NaN,8.0,SHARED2FREE,-1.686592,school,1,1,1,1,mandatory,1,636.0,641.0,1735309,61.0,8.0,15.0,7.0,NaN,NaN,SHARED3FREE,-0.969775,NaN,NaN,1out_0in,school,0.0,0,1,0,male,3,1,1735309,0.0,5,19 and under,1.0,633,6,asian,0,3,-1,no,2,0,641,8,4,1,-121.80644,37.350304,False,False,False,True,False,False,False,True,False,False,True,False,False,False,False,True,True,False,False,1,False,641,19.798008,636,1.826168,29.766666,-1,NaN,NaN,False,NaN,0.00,0.0000,0.0,False,M,True,False,False,False,school1,Fal

In [18]:
tourTripsMerged[tourTripsMerged['trip_id'] == 1306]

,trip_id,person_id,household_id,tour_id,primary_purpose_x,trip_num,outbound,trip_count,purpose,destination_x,origin_x,destination_logsum_x,depart,trip_mode,mode_choice_logsum_x,tour_type,tour_type_count,tour_type_num,tour_num,tour_count,tour_category,number_of_participants,destination_y,origin_y,household_id_x,tdd,start,end,duration,composition,destination_logsum_y,tour_mode,mode_choice_logsum_y,atwork_subtour_frequency,parent_tour_id,stop_frequency,primary_purpose_y,earning,worker,student,hispanic.1,person_sex,PNUM,sex,household_id_y,hours,MAR,person_age,edu,school_zone_id,race_id,race,work_at_home,age,work_zone_id,p_hispanic,relate,hispanic,TAZ_x,ptype,pemploy,pstudent,home_x,home_y,age_16_to_19,age_16_p,adult,male,female,has_non_worker,has_retiree,has_preschool_kid,has_driving_kid,has_school_kid,has_full_time,has_part_time,has_university,student_is_employed,nonstudent_to_school,is_student,is_gradeschool,is_highschool,is_university,school_segment,is_worker,home_taz,value_of_time,school_taz,distance_to_school,roundtrip_auto_time_to_school,workplace_taz,workplace_location_logsum,distance_to_work,workplace_in_cbd,work_taz_area_type,roundtrip_auto_time_to_work,work_auto_savings,work_auto_savings_ratio,free_parking_at_work,cdap_activity,travel_active,under16_not_at_school,has_preschool_kid_at_home,has_school_kid_at_home,mandatory_tour_frequency,work_and_school_and_worker,work_and_school_and_student,num_mand,num_work_tours,num_joint_tours,non_mandatory_tour_frequency,num_non_mand,num_escort_tours,num_eatout_tours,num_shop_tours,num_maint_tours,num_discr_tours,num_social_tours,num_non_escort_tours,gt55,seniors,VEHICL,lcm_county_id,hh_children,block_id,gt2,hispanic_head,age_of_head,race_of_head,tenure_mover,hh_size,sf_detached,tenure,hh_cars,income,hh_age_of_head,serialno,num_workers,hh_race_of_head,hh_income,recent_mover,hh_workers,hispanic_status_of_head,hh_seniors,hhsize,hh_type,TAZ_y,HHT,sample_rate,chunk_id,income_in_thousands,income_segment,median_value_of_time,hh_value_of_time,num_non_workers,num_drivers,num_adults,num_children,num_young_children,num_children_5_to_15,num_children_16_to_17,num_college_age,num_young_adults,non_family,family,home_is_urban,home_is_rural,auto_ownership,hh_work_auto_savings_ratio,num_under16_not_at_school,num_travel_active,num_travel_active_adults,num_travel_active_preschoolers,num_travel_active_children,num_travel_active_non_preschoolers,participates_in_jtf_model,joint_tour_frequency,num_hh_joint_tours,SERIALNO,SPORDER,AGEP,SEX,DIS,HINCP,VEH,JWMNP,JWRIP,JWTR,RAC1P,RAC2P05
11,1306,3,1735309,163,work,2,True,2,work,583,406,NaN,11.0,SHARED2PAY,-0.258008,work,2,2,2,2,mandatory,1,583.0,641.0,1735309,95.0,10.0,20.0,10.0,NaN,NaN,SHARED2PAY,0.076171,no_subtours,NaN,1out_0in,work,160000.0,1,0,0,male,1,1,1735309,40.0,1,20 to 35,21.0,-1,6,asian,0,35,583,no,0,0,641,1,1,3,-121.80644,37.350304,False,True,True,True,False,False,False,True,False,False,True,False,False,False,False,False,False,False,False,0,True,641,29.682171,-1,NaN,0.0,583,13.785309,1.781849,False,3.0,25.940277,29.419334,0.245161,False,M,True,False,False,False,work2,False,False,2,2,0,0,0,0,0,0,0,0,0,0,0,0.0,2,6085,yes,60855035091006,1,no,35,6,own not recent,four or more,yes,1,two or more,240000.0,gt35-lt65,2010000482865,2.0,asian,gt150,0,two or more,0,no,4,3,641,4,0.097,60138,240.0,4,12.86,29.682171,2.0,2,2,2,2,0,0,0,1,True,False,False,False,2,1.245161,0,4,2,2,2,2,True,0_tours,0,2010000482865,1,35,1,2,240000.0,2.0,20.0,1.0,1.0,6,40


In [127]:
# Merge trips, tours, households, and persons with land-use
#landuse = landuse.sort_values(by=['TAZ'])
#tourTripsMerged = tourTripsMerged.sort_values(by=['TAZ'])
#landuseTripsMerged = pd.merge(tourTripsMerged, landuse, how='left', left_on='TAZ', right_on='TAZ', suffixes=('', '_drop'))
#landuseTripsMerged.drop([col for col in landuseTripsMerged.columns if 'drop' in col], axis=1, inplace=True)

In [ ]:
# Concat mode_choice_raw files
path = "C:/Users/nazanin/Downloads/trip_mode_choice/trip_mode_choice/"
all_files = glob.glob(path + "*raw.csv")
li_mapper = map(lambda filename: pd.read_csv(filename, index_col = None, header = 0), all_files)
li2 = list(li_mapper)
SFmode_choice_raw = pd.concat(li2, axis = 0, ignore_index=True)

In [128]:
# Concat mode_choice_utilities files
path = "C:/Users/nazanin/Downloads/trip_mode_choice/trip_mode_choice/"
all_files = glob.glob(path + "*utilities.csv")
li_mapper = map(lambda filename: pd.read_csv(filename, index_col = None, header = 0), all_files)
li2 = list(li_mapper)
SFmode_choice_utilities = pd.concat(li2, axis = 0, ignore_index=True)

In [ ]:
# Merge mode_choice_raw and utilities
SFmode_choice_raw = SFmode_choice_raw.sort_values(by=['trip_id'])
SFmode_choice_utilities = SFmode_choice_utilities.sort_values(by=['trip_id'])
rawUtil = pd.merge(left=SFmode_choice_raw, right=SFmode_choice_utilities, how='left', on='trip_id')

In [129]:
%%time
#just utilities
# Merge trips, tours, households, persons, trip_mode_choice_raw, and utilities
tourTripsMerged = tourTripsMerged.sort_values(by=['trip_id'])
SFmode_choice_utilities = SFmode_choice_utilities.sort_values(by=['trip_id'])
SFActMerged= pd.merge(left=tourTripsMerged, right=SFmode_choice_utilities, how='left', on=['trip_id'])

CPU times: total: 11 s
Wall time: 11 s


In [ ]:
%%time
# Both raw and utilities
# Merge trips, tours, households, persons, trip_mode_choice_raw, and utilities
tourTripsMerged = tourTripsMerged.sort_values(by=['trip_id'])
rawUtil = rawUtil.sort_values(by=['trip_id'])
SFActMerged= pd.merge(left=tourTripsMerged, right=rawUtil, how='left', on=['trip_id'])

In [130]:
# Merge person_trip level BEAM with activity sim merged files
SFActMerged = SFActMerged.sort_values(by=['person_id', 'trip_id']).reset_index(drop=True)
Person_Trip_eventsSF = Person_Trip_eventsSF.sort_values(by=['IDMerged','tripIndex']).reset_index(drop=True)
eventsASim = pd.merge(left=Person_Trip_eventsSF, right=SFActMerged, how='left', left_on=["IDMerged", 'tripIndex'], right_on=['person_id', 'trip_id'])
#eventsASim = pd.merge(left=Person_Trip_eventsSF, right=tourTripsMerged, how='left',left_on = ["IDMerged", 'tripId'] , right_on=['person_id', 'trip_id'], suffixes=('', '_drop'))
#eventsASim.drop([col for col in eventsASim.columns if 'drop' in col], axis=1, inplace=True)

In [131]:
eventsASim.rename(columns={"mode_choice_logsum_y":"logsum_tours_mode_AS_tours"}, inplace=True)

In [132]:
eventsASim.rename(columns={"tour_mode":"tour_mode_AS_tours"}, inplace=True)

In [133]:
eventsASim.rename(columns={"mode_choice_logsum_x":"logsum_trip_mode_AS_trips"}, inplace=True)

In [134]:
eventsASim.rename(columns={"trip_mode":"trip_mode_AS_trips"}, inplace=True)

In [135]:
%%time
eventsASim.to_csv('s3://beam-core-act/deepDive/CleanData/SanFrancisco/Baseline/sf_2018_base_utilities.csv', index=False)

CPU times: total: 5min 49s
Wall time: 7min 25s


In [ ]:
eventsASim.to_csv('C:/Shared-Work/Data/CleanData/ASIM_BEAM_Merged/sf_2018_base.csv', index = False)

In [ ]:
# eventsASim.duration_in_privateCar.sum()

In [ ]:
eventsASim.head(50000).to_csv('C:/Shared-Work/Data/CleanData/ASIM_BEAM_Merged/sf_2018_base_test.csv', index = False)

In [ ]:
#import matplotlib.pyplot as plt
#import seaborn as sns

#plt.figure(figsize=(13, 6))
#sns.heatmap(households, vmax=1, annot=True, linewidths=.5)
#plt.xticks(rotation=30, horizontalalignment='right')
#plt.show()

In [ ]:
#import missingno as msno
#%matplotlib inline
#msno.bar(SFbaseline2018)
#msno.heatmap(persons)

In [ ]:
#from io import StringIO # python3; python2: BytesIO 
#import boto
#import boto3
#
#conn = boto.s3.connect_to_region('us-east-2')
#bucket = conn.get_bucket('beam-core-act')
#csv_buffer = StringIO()
#eventsASim.to_csv(csv_buffer)
#s3_resource = boto3.resource('s3')
#s3_resource.Object(bucket, 'eventsASim.csv').put(Body=csv_buffer.getvalue())